In [39]:
import pandas as pd
import json
import pymongo
from pprint import pprint 

# Let's first clean the CSV

In [60]:
df = pd.read_csv('15100005.csv')
df.tail()

,REF_DATE,GEO,DGUID,First official language spoken,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
691,2016,Canada outside Quebec,NaN,French,Number,Number,223,units,0,v1043024766,16.2.1,954810,NaN,NaN,NaN,0
692,2016,Canada outside Quebec,NaN,English,Number,Number,223,units,0,v1043024767,16.3.1,25043375,NaN,NaN,NaN,0
693,2016,Canada outside Quebec,NaN,English and french,Number,Number,223,units,0,v1043024768,16.4.1,138775,NaN,NaN,NaN,0
694,2016,Canada outside Quebec,NaN,Neither english nor French,Number,Number,223,units,0,v1043024769,16.5.1,563740,NaN,NaN,NaN,0
695,2016,Canada outside Quebec,NaN,Official language minority,Number,Number,223,units,0,v1043024770,16.6.1,1024200,NaN,NaN,NaN,0


In [61]:
#Find all of the values in the GEO column
df['GEO'].unique()

array(['Canada', 'Newfoundland and Labrador', 'Prince Edward Island',
       'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba',
       'Saskatchewan ', 'Alberta', 'British Columbia', 'Yukon',
       'Northwest Territories including Nunavut', 'Canada outside Quebec',
       'Northwest Territories', 'Nunavut'], dtype=object)

In [62]:
#Extract just the provinces
provinces = set(df['GEO'].unique()) - set(['Canada', 'Northwest Territories including Nunavut', 'Canada outside Quebec'])

In [63]:
# Get the data for just the provinces, also get rid of the other columns
df = df.loc[df['GEO'].isin(provinces), ['REF_DATE', 'GEO', 'First official language spoken', 'VALUE']]
df.head()

,REF_DATE,GEO,First official language spoken,VALUE
6,1971,Newfoundland and Labrador,"Total, first official language spoken",522104
7,1971,Newfoundland and Labrador,French,3638
8,1971,Newfoundland and Labrador,English,517736
9,1971,Newfoundland and Labrador,English and french,106
10,1971,Newfoundland and Labrador,Neither english nor French,624


In [64]:
# Clean up the dataset and pivot it to get unique columns
df = df.pivot(index = ['GEO', 'REF_DATE'], columns = 'First official language spoken', values = 'VALUE')
df.head(20) 

First official language spoken  English  English and french  French  \
GEO              REF_DATE                                             
Alberta          1971           1560209                2373   47301   
                 1981           2132396                7065   53253   
                 1991           2433560                5430   50565   
                 1996           2580675                6220   49390   
                 2001           2848810                6375   55645   
                 2006           3150170                8420   58575   
                 2011           3484240               12530   65105   
                 2016           3880680               16605   71535   
British Columbia 1971           2115130                3997   38769   
                 1981           2632280                8006   36090   
                 1991           3132795                8635   45225   
                 1996           3532485               12040   50285   
                 2001           3700385               14790   51970   
                 2006           3883215               17350   53060   
                 2011           4143245               16935   53725   
                 2016           4373335               18000   55320   
Manitoba         1971            904413                2146   61104   
                 1981            948317                3253   47459   
                 1991           1017500                2265   46665   
                 1996           1041230                2010   45570   

First official language spoken  Neither english nor French  \
GEO              REF_DATE                                    
Alberta          1971                                17991   
                 1981                                20937   
                 1991                                29635   
                 1996                                32910   
                 2001                                30360   
                 2006                                39185   
                 2011                                48310   
                 2016                                57830   
British Columbia 1971                                26725   
                 1981                                37239   
                 1991                                60850   
                 1996                                94945   
                 2001                               101725   
                 2006                               120760   
                 2011                               142300   
                 2016                               151760   
Manitoba         1971                                20584   
                 1981                                14675   
                 1991                                12965   
                 1996                                11490   

First official language spoken  Official language minority  \
GEO              REF_DATE                                    
Alberta          1971                                48488   
                 1981                                56785   
                 1991                                53280   
                 1996                                52500   
                 2001                                58833   
                 2006                                62790   
                 2011                                71370   
                 2016                                79840   
British Columbia 1971                                40768   
                 1981                                40093   
                 1991                                49545   
                 1996                                56310   
                 2001                                59365   
                 2006                                61735   
                 2011                                62190   
                 2

In [65]:
# Some additional clean up
column_names = ['province', 'date', 'english', 'english_and_french', 'french', 'neither_official', 'official_language_minority', 'total']

df = df.reset_index()
df.columns = column_names
df.head()

,province,date,english,english_and_french,french,neither_official,official_language_minority,total
0,Alberta,1971,1560209,2373,47301,17991,48488,1627874
1,Alberta,1981,2132396,7065,53253,20937,56785,2213651
2,Alberta,1991,2433560,5430,50565,29635,53280,2519180
3,Alberta,1996,2580675,6220,49390,32910,52500,2669195
4,Alberta,2001,2848810,6375,55645,30360,58833,2941190


In [69]:
# These numbers are great, but what about per/capita
# Of course we want to go back to 1971...

pop = pd.read_csv('1710000901-eng.csv', skiprows=5)
pop.head()

,Geography,Q1 1971,Q2 1971,Q3 1971,Q4 1971,Q1 1972,Q2 1972,Q3 1972,Q4 1972,Q1 1973,...,Q2 2014,Q3 2014,Q4 2014,Q1 2015,Q2 2015,Q3 2015,Q4 2015,Q1 2016,Q2 2016,Q3 2016
0,NaN,Persons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Canada,"21,465,000","21,523,000","21,962,032","22,039,243","22,092,498","22,148,919","22,218,463","22,288,499","22,347,459",...,"35,323,533","35,437,435","35,559,047","35,575,187","35,611,271","35,702,908","35,822,894","35,871,136","35,970,303","36,109,487"
2,Newfoundland and Labrador,"519,000","521,000","530,854","534,471","535,736","537,368","539,124","541,472","542,983",...,"527,498","528,159","528,461","528,361","528,063","528,117","528,696","528,800","529,063","529,426"
3,Prince Edward Island,"111,000","111,000","112,591","112,874","112,964","113,314","113,460","113,960","114,276",...,"143,984","144,283","144,538","144,368","144,259","144,546","144,894","145,202","145,784","146,969"
4,Nova Scotia,"785,000","788,000","797,294","798,804","800,513","800,542","802,255","805,317","808,614",...,"938,913","938,545","938,972","938,296","936,271","936,525","938,320","938,690","940,382","942,790"


In [71]:
pop = pop.loc[pop['Geography'].isin(geo_provinces)]

In [84]:
# Extract only Q1
q1s = pop.columns[pop.columns.str.contains('Q1')]
q1s

Index(['Q1 1971', 'Q1 1972', 'Q1 1973', 'Q1 1974', 'Q1 1975', 'Q1 1976',
       'Q1 1977', 'Q1 1978', 'Q1 1979', 'Q1 1980', 'Q1 1981', 'Q1 1982',
       'Q1 1983', 'Q1 1984', 'Q1 1985', 'Q1 1986', 'Q1 1987', 'Q1 1988',
       'Q1 1989', 'Q1 1990', 'Q1 1991', 'Q1 1992', 'Q1 1993', 'Q1 1994',
       'Q1 1995', 'Q1 1996', 'Q1 1997', 'Q1 1998', 'Q1 1999', 'Q1 2000',
       'Q1 2001', 'Q1 2002', 'Q1 2003', 'Q1 2004', 'Q1 2005', 'Q1 2006',
       'Q1 2007', 'Q1 2008', 'Q1 2009', 'Q1 2010', 'Q1 2011', 'Q1 2012',
       'Q1 2013', 'Q1 2014', 'Q1 2015', 'Q1 2016'],
      dtype='object')

In [77]:
# We also only have a subset of years
years = df['date'].unique()
years

array([1971, 1981, 1991, 1996, 2001, 2006, 2011, 2016])

In [83]:
mask = [True if i in years else False for i in range(1971, 2017)]

In [88]:
keep_cols = list(q1s[mask]) + ['Geography']

In [91]:
pop = pop[keep_cols]
pop.head() 

,Q1 1971,Q1 1981,Q1 1991,Q1 1996,Q1 2001,Q1 2006,Q1 2011,Q1 2016,Geography
2,"519,000","573,420","577,377","563,679","525,299","512,564","523,726","528,800",Newfoundland and Labrador
3,"111,000","123,315","130,477","135,119","136,377","137,907","143,020","145,202",Prince Edward Island
4,"785,000","853,673","912,792","929,782","933,463","938,117","944,068","938,690",Nova Scotia
5,"630,000","704,763","743,210","751,581","749,715","746,983","754,794","760,580",New Brunswick
6,"6,017,000","6,523,258","7,026,241","7,232,952","7,373,978","7,604,258","7,967,185","8,193,624",Quebec


In [94]:
pop.columns = [i.replace('Q1 ', '') for i in keep_cols]

In [95]:
pop

,1971,1981,1991,1996,2001,2006,2011,2016,Geography
2,"519,000","573,420","577,377","563,679","525,299","512,564","523,726","528,800",Newfoundland and Labrador
3,"111,000","123,315","130,477","135,119","136,377","137,907","143,020","145,202",Prince Edward Island
4,"785,000","853,673","912,792","929,782","933,463","938,117","944,068","938,690",Nova Scotia
5,"630,000","704,763","743,210","751,581","749,715","746,983","754,794","760,580",New Brunswick
6,"6,017,000","6,523,258","7,026,241","7,232,952","7,373,978","7,604,258","7,967,185","8,193,624",Quebec
7,"7,656,000","8,770,591","10,355,101","11,009,307","11,771,945","12,587,531","13,198,650","13,773,629",Ontario
8,"984,000","1,033,968","1,106,196","1,130,288","1,148,456","1,179,920","1,227,212","1,302,544",Manitoba
9,"927,000","971,544","1,002,651","1,016,198","1,003,469","992,237","1,058,963","1,127,773",Saskatchewan
10,"1,616,000","2,249,907","2,572,947","2,753,463","3,027,941","3,371,823","3,754,424","4,169,830",Alberta
11,"2,168,000","2,790,754","3,339,935","3,826,739","4,055,229","4,217,962","4,482,260","4,811,465",British Columbia


In [99]:
pop = pop.applymap(lambda x: x.replace(',', '').replace('..', '0'))

In [100]:
pop

,1971,1981,1991,1996,2001,2006,2011,2016,Geography
2,519000,573420,577377,563679,525299,512564,523726,528800,Newfoundland and Labrador
3,111000,123315,130477,135119,136377,137907,143020,145202,Prince Edward Island
4,785000,853673,912792,929782,933463,938117,944068,938690,Nova Scotia
5,630000,704763,743210,751581,749715,746983,754794,760580,New Brunswick
6,6017000,6523258,7026241,7232952,7373978,7604258,7967185,8193624,Quebec
7,7656000,8770591,10355101,11009307,11771945,12587531,13198650,13773629,Ontario
8,984000,1033968,1106196,1130288,1148456,1179920,1227212,1302544,Manitoba
9,927000,971544,1002651,1016198,1003469,992237,1058963,1127773,Saskatchewan
10,1616000,2249907,2572947,2753463,3027941,3371823,3754424,4169830,Alberta
11,2168000,2790754,3339935,3826739,4055229,4217962,4482260,4811465,British Columbia


In [102]:
def conv(x):
    try:
        return int(x)
    except:
        return x

pop = pop.applymap(conv)

In [105]:
pop

,1971,1981,1991,1996,2001,2006,2011,2016,Geography
2,519000,573420,577377,563679,525299,512564,523726,528800,Newfoundland and Labrador
3,111000,123315,130477,135119,136377,137907,143020,145202,Prince Edward Island
4,785000,853673,912792,929782,933463,938117,944068,938690,Nova Scotia
5,630000,704763,743210,751581,749715,746983,754794,760580,New Brunswick
6,6017000,6523258,7026241,7232952,7373978,7604258,7967185,8193624,Quebec
7,7656000,8770591,10355101,11009307,11771945,12587531,13198650,13773629,Ontario
8,984000,1033968,1106196,1130288,1148456,1179920,1227212,1302544,Manitoba
9,927000,971544,1002651,1016198,1003469,992237,1058963,1127773,Saskatchewan
10,1616000,2249907,2572947,2753463,3027941,3371823,3754424,4169830,Alberta
11,2168000,2790754,3339935,3826739,4055229,4217962,4482260,4811465,British Columbia


In [111]:
pop = pop.set_index('Geography').stack().reset_index()

In [116]:
pop.columns = ['province', 'date', 'population']

In [128]:
pop['date'] = pd.to_numeric(pop['date'])

In [129]:
pop.dtypes

province      object
date           int64
population     int64
dtype: object

In [133]:
comb = df.merge(pop, on=['province', 'date'])

In [134]:
comb['english-pct'] = comb['english'] / comb['population']
comb['french-pct'] = comb['french'] / comb['population']
comb['neither-pct'] = comb['neither_official'] / comb['population']

In [135]:
comb

,province,date,english,english_and_french,french,neither_official,official_language_minority,total,population,english-pct,french-pct,neither-pct
0,Alberta,1971,1560209,2373,47301,17991,48488,1627874,1616000,0.965476,0.029270,0.011133
1,Alberta,1981,2132396,7065,53253,20937,56785,2213651,2249907,0.947771,0.023669,0.009306
2,Alberta,1991,2433560,5430,50565,29635,53280,2519180,2572947,0.945826,0.019653,0.011518
3,Alberta,1996,2580675,6220,49390,32910,52500,2669195,2753463,0.937247,0.017937,0.011952
4,Alberta,2001,2848810,6375,55645,30360,58833,2941190,3027941,0.940841,0.018377,0.010027
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Yukon,1996,29455,70,1080,45,1115,30655,30963,0.951297,0.034880,0.001453
84,Yukon,2001,27545,60,850,60,880,28515,30147,0.913690,0.028195,0.001990
85,Yukon,2006,28830,120,1120,120,1185,30195,32112,0.897795,0.034878,0.003737
86,Yukon,2011,32015,125,1420,95,1485,33655,34801,0.919945,0.040803,0.002730


# Let's Deal with the geoJSON data

In [26]:
with open('canada_provinces.geojson') as f:
    d = f.read()
    data = json.loads(d)

In [30]:
geo_provinces = [prov['properties']['name'] for prov in data['features']]

In [33]:
geo_provinces

['British Columbia',
 'Newfoundland and Labrador',
 'Northwest Territories',
 'Nunavut',
 'Saskatchewan',
 'Quebec',
 'Alberta',
 'Manitoba',
 'Nova Scotia',
 'Ontario',
 'New Brunswick',
 'Prince Edward Island',
 'Yukon']

In [34]:
provinces

{'Alberta',
 'British Columbia',
 'Manitoba',
 'New Brunswick',
 'Newfoundland and Labrador',
 'Northwest Territories',
 'Nova Scotia',
 'Nunavut',
 'Ontario',
 'Prince Edward Island',
 'Quebec',
 'Saskatchewan ',
 'Yukon'}

In [32]:
provinces - set(geo_provinces)

{'Saskatchewan '}

In [35]:
df['province'] = df['province'].map(lambda x: x.strip())

In [36]:
df['province'].unique()

array(['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Northwest Territories',
       'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island',
       'Quebec', 'Saskatchewan', 'Yukon'], dtype=object)

In [141]:
for prov in data['features']:
    prov_df = comb.loc[df['province'] == prov['properties']['name']].copy()

    years = prov_df['date'].unique()

    for year in years:
        prov['properties'][f'english-{year}'] = float(prov_df.loc[prov_df['date'] == year, 'english-pct'].values[0])
        prov['properties'][f'french-{year}'] = float(prov_df.loc[prov_df['date'] == year, 'french-pct'].values[0])
        prov['properties'][f'neither-{year}'] = float(prov_df.loc[prov_df['date'] == year, 'neither-pct'].values[0])

In [142]:
[pprint(prov['properties']) for prov in data['features']]

{'_2012_membership': 39859,
 '_end': None,
 'altitudemode': 'relativeToGround',
 'begin': None,
 'cartodb_id': 6,
 'description': None,
 'draworder': None,
 'english-1971': 0.9756134686346863,
 'english-1981': 0.9432146294513956,
 'english-1991': 0.9379808289682284,
 'english-1996': 0.9231058088884557,
 'english-2001': 0.9124971734025378,
 'english-2006': 0.9206377392683955,
 'english-2011': 0.9243651640020882,
 'english-2016': 0.9089404162765394,
 'extrude': -1,
 'french-1971': 0.01788238007380074,
 'french-1981': 0.012931988989355565,
 'french-1991': 0.013540682677956307,
 'french-1996': 0.013140431056311914,
 'french-2001': 0.012815552463251768,
 'french-2006': 0.012579534855932793,
 'french-2011': 0.011986140919982331,
 'french-2016': 0.011497537652253524,
 'icon': None,
 'name': 'British Columbia',
 'neither-1971': 0.012327029520295203,
 'neither-1981': 0.013343705679540368,
 'neither-1991': 0.01821891743402192,
 'neither-1996': 0.02481094216250442,
 'neither-2001': 0.025084896561

[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [41]:
#Let's save the data
with open('provinces_language.json', 'w') as f: 
    f.write(json.dumps(data))